In [1]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.python.keras import applications
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from tensorflow.python.keras.optimizers import RMSprop
import numpy as np
import pandas as pd

In [2]:
#parameters
classes = 12
batch_size=32
train_total = 3783
validation_total = 967

In [3]:
model_resnet = ResNet50(include_top=False, weights = 'imagenet',input_shape=(224,224,3))

In [11]:
model_resnet.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        input_3[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_99 (Activation)       (None, 112, 112, 64)  0           bn_conv1[0][0]                   
___________________________________________________________________________________________

In [10]:
x = model_resnet.get_layer('avg_pool').output
x = Flatten()(x)
x = Dense(classes, activation='softmax')(x)

In [11]:
model = Model(inputs=model_resnet.input, outputs=x)

In [14]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        input_1[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 112, 112, 64)  0           bn_conv1[0][0]                   
___________________________________________________________________________________________

In [ ]:
model.get_layer('')

In [13]:
for layer in model_resnet.layers:
    layer.trainable= False

In [15]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [16]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(224,224),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(224,224),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 3783 images belonging to 12 classes.
Found 967 images belonging to 12 classes.


In [40]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=0.000005),
             metrics=['accuracy'])

In [41]:
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=50,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2)

Epoch 1/50
91s - loss: 0.5310 - acc: 0.8335 - val_loss: 0.4770 - val_acc: 0.8418
Epoch 2/50
90s - loss: 0.5390 - acc: 0.8302 - val_loss: 0.5011 - val_acc: 0.8490
Epoch 3/50
90s - loss: 0.5297 - acc: 0.8357 - val_loss: 0.5004 - val_acc: 0.8397
Epoch 4/50
91s - loss: 0.5308 - acc: 0.8337 - val_loss: 0.4853 - val_acc: 0.8438
Epoch 5/50
86s - loss: 0.5310 - acc: 0.8290 - val_loss: 0.5000 - val_acc: 0.8438
Epoch 6/50
87s - loss: 0.5265 - acc: 0.8404 - val_loss: 0.4970 - val_acc: 0.8376
Epoch 7/50
88s - loss: 0.5359 - acc: 0.8303 - val_loss: 0.5140 - val_acc: 0.8314
Epoch 8/50
86s - loss: 0.5454 - acc: 0.8255 - val_loss: 0.4976 - val_acc: 0.8418
Epoch 9/50
87s - loss: 0.5417 - acc: 0.8304 - val_loss: 0.5009 - val_acc: 0.8366
Epoch 10/50
91s - loss: 0.5408 - acc: 0.8160 - val_loss: 0.4900 - val_acc: 0.8418
Epoch 11/50
88s - loss: 0.5447 - acc: 0.8210 - val_loss: 0.4999 - val_acc: 0.8438
Epoch 12/50
87s - loss: 0.5284 - acc: 0.8298 - val_loss: 0.5018 - val_acc: 0.8345
Epoch 13/50
89s - loss: 0

In [38]:
model.save_weights('resnet50_bottom_fixed_tweak.hdf5')

In [9]:
model.load_weights('resnet50_bottom_fixed_tweak.hdf5')

In [54]:
model.layers.count()

<function list.count>

In [29]:
from IPython.display import FileLink
FileLink('resnet50_bottom_fixed.hdf5')

/home/janardhan/plant_seeding/resnet50_bottom_fixed.hdf5